#### Peruse Results

In [1]:
#packages
import os

import numpy as np
import pandas as pd

import torch
import torchvision



#settings
res_dir = "/scratch/ym2380/data/celebA/results"

In [2]:
def calc_helper(group_id, indices, indices_cor):
    n_corr = len(indices_cor)
    n = len(indices)
    #print(f"{group_id}: n_corr {n_corr} n_wrong {n - n_corr} ratio {n_corr / n:.4f}")
    return n - n_corr
    

def peruse_res(fname, conditional_epoch):
    df = pd.read_csv(os.path.join(res_dir, fname))
    
    res_col = df[f"e{conditional_epoch:02d}"]
    target_col = df["Blond_Hair"]
    
    edf = pd.read_csv(os.path.join("/scratch/ym2380/data/celebA/data", "list_attr_celeba.csv"))
    edf.rename(columns={"Blond_Hair": "Blond_Hair_backup"}, inplace=True)
    mdf = pd.merge(df, edf, on="image_id")
    mdf["Male"] = (mdf["Male"] + 1) / 2
    mdf["Blond_Hair_backup"] = (mdf["Blond_Hair_backup"] + 1) / 2
    
    spurious_col = mdf["Male"]
    #debug
    assert (mdf["Blond_Hair_backup"] == target_col).all()
    
    condition11 = (target_col == 1) & (spurious_col == 1)
    condition10 = (target_col == 1) & (spurious_col == 0)
    condition01 = (target_col == 0) & (spurious_col == 1)
    condition00 = (target_col == 0) & (spurious_col == 0)
    
    indices11 = df.index[condition11]
    indices10 = df.index[condition10]
    indices01 = df.index[condition01]
    indices00 = df.index[condition00]
    
    indices_cor11 = df.index[condition11 & (res_col == target_col)]
    indices_cor10 = df.index[condition10 & (res_col == target_col)]
    indices_cor01 = df.index[condition01 & (res_col == target_col)]
    indices_cor00 = df.index[condition00 & (res_col == target_col)]
    
    n_wrong_11 = calc_helper("11", indices11, indices_cor11)
    n_wrong_10 = calc_helper("10", indices10, indices_cor10)
    n_wrong_01 = calc_helper("01", indices01, indices_cor01)
    n_wrong_00 = calc_helper("00", indices00, indices_cor00)

    return {"filename": fname, "epoch": conditional_epoch, "11": n_wrong_11, "10": n_wrong_10, "01": n_wrong_01, "00": n_wrong_00}

def peruse_directory(directory=res_dir):
    df = pd.DataFrame(columns=["filename", "epoch", "11", "10", "01", "00"])
    
    fname_list = os.listdir(directory)
    for fname in fname_list:
        for i in range(1, 11):
            try:
                row_dict = peruse_res(fname, i)
                df.loc[len(df)] = row_dict
            except Exception:
                print(f"filename {fname} no epoch {i}")

    return df

In [3]:
df = peruse_directory()

filename 06070415_15 no epoch 10
filename 06070415_22 no epoch 10
filename 06070415_11 no epoch 10
filename 06070617_4 no epoch 7
filename 06070617_4 no epoch 8
filename 06070617_4 no epoch 9
filename 06070617_4 no epoch 10
filename 06070415_8 no epoch 10
filename 06070415_18 no epoch 10
filename 06070415_23 no epoch 10
filename 06070415_9 no epoch 10
filename 06070415_10 no epoch 10
filename 06070415_7 no epoch 10


In [ ]:
pd.set_option("display.max_rows", 1000)

print(len(df))
df

In [45]:
ndf = df

ndf = ndf[ 
    (ndf["11"] < 1200) &
    (ndf["11"] > 900) &
    (ndf["10"] < 6000) & 
    (ndf["10"] > 4500) &
    (ndf["01"] > 600) &
    (ndf["01"] < 1200) &
    (ndf["00"] > 0) &
    (ndf["00"] < 3000)
]

print(len(ndf))
ndf

5


,filename,epoch,11,10,01,00
194,06070508_4,5,907,5439,678,2694
284,06070508_3,5,901,5202,650,2820
401,06070617_2,2,909,5879,695,2998
660,06070508_11,2,949,5844,711,2824
749,06070508_6,3,946,5862,665,2858


In [34]:
pd.set_option("display.max_rows", 1000)
len(df)
df

,filename,epoch,11,10,01,00
0,06070415_3,1,911,7918,1296,2867
1,06070415_3,2,906,5110,678,3065
2,06070415_3,3,948,4528,494,3124
3,06070415_3,4,932,4326,435,3081
4,06070415_3,5,946,4085,429,3083
5,06070415_3,6,952,3939,381,3154
6,06070415_3,7,916,3726,386,3245
7,06070415_3,8,947,3821,370,3169
8,06070415_3,9,929,3769,336,3183
9,06070415_3,10,945,3849,341,3069
